# In which the author attempts to justify his aesthetic vision of code and tests thereof

Ok, at this point, Harry is heading off criticism and concerns about the plodding pace of the unit tests introduced in Chapter 3.  He implies that if you have such feedback, you're far too naive about writing maintainable code.  He suggests building discipline in forcing yourself to follow the model of making *really* incremental changes whenever you apply TDD; the idea is to minimize the amount of work you have to re-do whenever your code fails.  If you break it up into small enough pieces, and test each separately, addressing each (inevitable) failure becomes a lot simpler.

Ok, then we resume adding stuff to `functional_tests.py` to address the issue encountered at the end of the last chapter where there was just a placeholder print statement reminding us that we hadn't done that yet.

In [3]:
%%writefile functional_tests.py
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import unittest

class NewVisitorTest(unittest.TestCase):

    def setUp(self):
        self.browser = webdriver.Firefox()

    def tearDown(self):
        self.browser.quit()

    def test_can_start_a_list_and_retrieve_it_later(self):
        # Edith has heard about a cool new online to-do app.  She goes
        # to check out its homepage
        self.browser.get('http://localhost:8000')

        # She notices the page title and header mention to-do lists
        self.assertIn('To-Do', self.browser.title)
        header_text = self.browser.find_element_by_tag_name('h1').text
        self.assertIn('To-Do', header_text)

        # She is invited to enter a to-do item straight away
        inputbox = self.browser.find_element_by_id('id_new_item')
        self.assertEqual(
            inputbox.get_attribute('placeholder'),
            'Enter a to-do item'
        )

        # She types "Buy peacock feathers" into a text box 
        # (Edith's hobby is tying fly-fishing lures)
        inputbox.send_keys('Buy peacock feathers')

        # When she hits enter, the page updates, and now the page lists
        # "1: Buy peacock feathers" as an item in a to-do list table
        inputbox.send_keys(Keys.ENTER)
        time.sleep(1)

        table = self.browser.find_element_by_id('id_list_table')
        rows = table.find_elements_by_tag_name('tr')
        self.assertTrue(
            any(row.text == '1: Buy peacock feathers' for row in rows)
        )

        # There is still a text box inviting her to add another item.
        # Shee enters "Use peacock feathers to make a fly"
        # (Edith is very methodical)
        self.fail('Finish the test!')

        # The page updates again, and now shows both items on her list
        # ...

if __name__ == '__main__':
    unittest.main(warnings='ignore')


Overwriting functional_tests.py


Now, I'm just a humble passive-aggressive misanthrope with a bum knee and an array of interesting sub-clinical manifestations of various other personality disorders, but to me that looks a lot *less* incremental than was preached so far.  So we'll see how he plays it.

First, we summarize the main effects of the above:

1. It introduces a few major functions from Selenium's `selenium.browser` object: `find_element_by_tag_name`, and so forth.
2. We point out that the `selenium.browser.send_keys` is the main way to simulate user input to Selenium
3. To use `send_keys`, though, we also have to import `selenium.webdriver.common.keys.Keys`, to access all the major QWERTY keyboard keys that you may need
4. The page should refresh after the `send_keys(Keys.ENTER)` command (line 38), so that `time.sleep` call is meant to give the server time to catch up before testing the assertions below; apparently this is called an "*explicit wait*"

Then, fire up the dev server and run the updated functional tests:

In [4]:
%run functional_tests

E
ERROR: test_can_start_a_list_and_retrieve_it_later (__main__.NewVisitorTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "c:\Data\projects\test_driven_development\functional_tests.py", line 21, in test_can_start_a_list_and_retrieve_it_later
    header_text = self.browser.find_element_by_tag_name('h1').text
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 530, in find_element_by_tag_name
    return self.find_element(by=By.TAG_NAME, value=name)
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 978, in find_element
    'value': value})['value']
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 321, in execute
    self.error_handler.check_response(response)
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\

SystemExit: True

Ok, so it can't find an `h1` tag in the loaded HTML.  But at this point, he recommends another commit, to store the changes to the `functional_tests.py` file.